# Randomized Projection on the Sensor dataset

scitkit learn random projection: https://scikit-learn.org/stable/modules/random_projection.html

## Load libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn import random_projection
from sklearn.preprocessing import normalize
from sklearn.neural_network import MLPClassifier

## Set directories

In [2]:
directory_hw1 = "/Users/mikepecorino/Documents/machine_learning/HW1/"
directory_hw3 = "/Users/mikepecorino/Documents/machine_learning/HW3/"

## Load inputs

In [3]:
data_all = pd.read_csv(directory_hw3 + "sensor_all.csv")

## Train/Test split

In [4]:
train = data_all[data_all["tag"].isin(["train", "valid"])]
test = data_all[data_all["tag"] == "test"]

## Define features and response variable

### Features list

In [5]:
features = data_all.columns[data_all.columns.isin(["subject", "activity_raw", "activity", "tag", "fold", "response_prop"]) == False]

### Features data

In [6]:
data_all_features = data_all[features]
train_features = train[features]
test_features = test[features]

### Response variable

In [7]:
response = "activity"

### Response data

In [8]:
data_all_response = data_all[response]
train_response = train[response]
test_response = test[response]

## Normalize data

In [9]:
data_all_features_normalized = normalize(data_all_features)
train_features_normalized = normalize(train_features)
test_features_normalized = normalize(test_features)

## Get best neural net hyperparameters from HW1

In [10]:
neural_network_results = pd.read_csv(directory_hw1 + "sensor_neural_net_grid_search.csv")
out_fold_cv_summary = neural_network_results.groupby(["hidden_layer_size", "solver", "activation", "alpha"])["out_fold_accuracy"].agg(["mean"]).sort_values(by = ["mean"], ascending = False)
optimal_results = out_fold_cv_summary.index[np.argmax(out_fold_cv_summary["mean"])]
hidden_layer_size_opt = optimal_results[0]
solver_opt = optimal_results[1]
activation_opt = optimal_results[2]
alpha_opt = optimal_results[3]

## Optimizing the number of components

In [11]:
#Initialize an empty data frame for recording results
neural_network_results = pd.DataFrame(columns = ["iter_counter",
                                                 "hidden_layer_size",
                                                 "solver",
                                                 "activation",
                                                 "alpha",
                                                 "train_accuracy",
                                                 "test_accuracy",
                                                 "time"
                                                 ])

#Set hyperparameter space
n_components = range(10, 610, 50)

#Start an iteration counter
iter_counter = 0

#For each alpha, hidden layer size, and fold...
#(using fixed solver and activiation function for now)
for n_component in n_components:
    
    #Increment the counter
    iter_counter = iter_counter + 1
    
    #Output message
    print("Iter:", iter_counter,
            "| number of components:", n_component)

    #Get the random projection
    start_time = time.time()
    
    transformer = random_projection.GaussianRandomProjection(n_components = n_component,
                                                             eps = 0.1,
                                                             random_state = 28)
    train_features_normalized_randpro = transformer.fit_transform(train_features_normalized)
    test_features_normalized_randpro = transformer.fit_transform(test_features_normalized)

    #Get the neural net object
    neural_network = MLPClassifier(solver = solver_opt, #lbfgs, adam, sgd
                                   activation = activation_opt, #identity, logistic, tanh, relu
                                   alpha = alpha_opt,
                                   hidden_layer_sizes = (hidden_layer_size_opt,),
                                   batch_size = "auto",
                                   learning_rate = "constant",
                                   learning_rate_init = 0.001,
                                   power_t = 0.5,
                                   max_iter = 200,
                                   shuffle = True,
                                   random_state = 28,
                                   tol = 0.0001,
                                   verbose = False,
                                   warm_start = False,
                                   momentum = 0.9,
                                   nesterovs_momentum = True,
                                   early_stopping = True,
                                   validation_fraction = 0.1,
                                   beta_1 = 0.9,
                                   beta_2 = 0.999,
                                   epsilon = 1e-08,
                                   n_iter_no_change = 10,
                                   max_fun = 15000)
    
    #Fit the model
    neural_network.fit(train_features_normalized_randpro, train_response) 
    
    #Evaluate the model
    #In fold
    train_accuracy = neural_network.score(train_features_normalized_randpro, train_response)
    #Out of fold
    test_accuracy = neural_network.score(test_features_normalized_randpro, test_response)

    end_time = time.time()
    total_time = end_time - start_time

    #Add to results list
    neural_network_results = neural_network_results.append({"iter_counter": iter_counter,
                                                            "n_components": n_component,
                                                            "hidden_layer_size": hidden_layer_size_opt,
                                                            "solver": solver_opt,
                                                            "activation": activation_opt,
                                                            "alpha": alpha_opt,
                                                            "train_accuracy": train_accuracy,
                                                            "test_accuracy": test_accuracy,
                                                            "time": total_time},
                                                            ignore_index = True)
    print(neural_network_results, "\n")
    neural_network_results.to_csv(directory_hw3 + "sensor_rp_neural_net_optimization.csv", index = False)

Iter: 1 | number of components: 10
  iter_counter hidden_layer_size solver activation  alpha  train_accuracy  \
0            1              1000   adam       relu   0.02        0.775843   

   test_accuracy      time  n_components  
0       0.743129  4.636758          10.0   

Iter: 2 | number of components: 60
  iter_counter hidden_layer_size solver activation  alpha  train_accuracy  \
0            1              1000   adam       relu   0.02        0.775843   
1            2              1000   adam       relu   0.02        0.957018   

   test_accuracy      time  n_components  
0       0.743129  4.636758          10.0  
1       0.921276  7.600787          60.0   

Iter: 3 | number of components: 110
  iter_counter hidden_layer_size solver activation  alpha  train_accuracy  \
0            1              1000   adam       relu   0.02        0.775843   
1            2              1000   adam       relu   0.02        0.957018   
2            3              1000   adam       relu   0.02

   iter_counter hidden_layer_size solver activation  alpha  train_accuracy  \
0             1              1000   adam       relu   0.02        0.775843   
1             2              1000   adam       relu   0.02        0.957018   
2             3              1000   adam       relu   0.02        0.958379   
3             4              1000   adam       relu   0.02        0.980413   
4             5              1000   adam       relu   0.02        0.978645   
5             6              1000   adam       relu   0.02        0.976061   
6             7              1000   adam       relu   0.02        0.978781   
7             8              1000   adam       relu   0.02        0.976877   
8             9              1000   adam       relu   0.02        0.977557   
9            10              1000   adam       relu   0.02        0.979189   
10           11              1000   adam       relu   0.02        0.972252   

    test_accuracy       time  n_components  
0        0.743129 